# Cleaned Data Merge and Upload to S3

##  Part I: Obtain the Weeks-on-board

In [5]:
import pandas as pd

In [6]:
songs = pd.read_csv('billboard.csv')

In [7]:
songs.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18


In [8]:
df_0 = songs[songs['song'] == 'Easy On Me']

In [9]:
max_val = songs[songs['song'] == 'Easy On Me']['weeks-on-board'].max()

In [32]:
for i in range(1958, 2022):
    songs_each_year = pd.read_csv(r'{}.csv'.format(i))
    lst = []
    for j in range(songs_each_year.shape[0]):
        max_val = songs[songs['song'] == songs_each_year.iloc[j, 3]]['weeks-on-board'].max()
        lst.append(max_val)
    songs_each_year['weeks-on-board'] = lst
    songs_each_year.to_csv(f'{i}_rank.csv', index=False)

## Part II: Upload to the S3

In [9]:
import boto3

In [10]:
s3 = boto3.client('s3')
s3_resource = boto3.resource('s3')

In [11]:
bucket = s3.create_bucket(Bucket='final-project-raw-data')

In [12]:
# check if it's already there
bucket_response = s3.list_buckets()
buckets = bucket_response['Buckets']
print(buckets)

[{'Name': 'aws-emr-resources-336011647036-us-east-1', 'CreationDate': datetime.datetime(2022, 5, 24, 21, 32, 40, tzinfo=tzutc())}, {'Name': 'aws-logs-336011647036-us-east-1', 'CreationDate': datetime.datetime(2022, 5, 24, 21, 32, 40, tzinfo=tzutc())}, {'Name': 'final-project-raw-data', 'CreationDate': datetime.datetime(2022, 6, 2, 0, 59, 46, tzinfo=tzutc())}, {'Name': 'survey-bao', 'CreationDate': datetime.datetime(2022, 5, 3, 21, 7, 13, tzinfo=tzutc())}]


In [13]:
for i in range(1958, 2022):
    path = f'{i}_rank.csv'
    s3.upload_file(Filename=path,
       Bucket='final-project-raw-data',
       Key = f'lyrics_{i}.csv')

In [14]:
# get response

response = s3.list_objects(
            Bucket='final-project-raw-data',
            MaxKeys=20)
print(len(response['Contents']))

20


In [15]:
response['Contents']

[{'Key': 'lyrics_1958.csv',
  'LastModified': datetime.datetime(2022, 6, 2, 6, 34, 4, tzinfo=tzutc()),
  'ETag': '"1aec2716ffc894bec2deb5df780cc38a"',
  'Size': 35445,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': 'awslabsc0w2793617t1631881177',
   'ID': 'cc9ffde1bc7e4669af93a17c249880be976cf82cf77dfc0bc75ed252ac8ac9e5'}},
 {'Key': 'lyrics_1959.csv',
  'LastModified': datetime.datetime(2022, 6, 2, 6, 34, 4, tzinfo=tzutc()),
  'ETag': '"b407f914b3d7edd88ea5488a6419abf5"',
  'Size': 47478,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': 'awslabsc0w2793617t1631881177',
   'ID': 'cc9ffde1bc7e4669af93a17c249880be976cf82cf77dfc0bc75ed252ac8ac9e5'}},
 {'Key': 'lyrics_1960.csv',
  'LastModified': datetime.datetime(2022, 6, 2, 6, 34, 5, tzinfo=tzutc()),
  'ETag': '"d6e6dd71d03dfffab94cc62672967cff"',
  'Size': 58146,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': 'awslabsc0w2793617t1631881177',
   'ID': 'cc9ffde1bc7e4669af93a17c249880be976cf82cf77dfc0bc75ed252ac8ac